In [ ]:
pip install --quiet rdflib SPARQLWrapper pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.6 MB/s eta 0:00:00


In [ ]:
import rdflib
import ssl
import json
import pprint
import pandas as pd
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import URIRef

ssl._create_default_https_context = ssl._create_unverified_context
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql_wd = SPARQLWrapper(wikidata_endpoint)
sparql_wd.setReturnFormat(JSON)

In [ ]:
query = """
SELECT (COUNT(DISTINCT ?person) as ?count) WHERE {
  ?person wdt:P31 wd:Q5.
  ?person wdt:P106 wd:Q484188.
}
"""

sparql_wd.setQuery(query)
sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Total Serial Killers': result['count']['value']
    }
    for result in results['results']['bindings']
])

print(results)

  Total Serial Killers
0                 1099


In [ ]:
query = """
SELECT ?serialKillerLabel ?victimCount ?countryOfCitizenshipLabel WHERE {
?serialKiller wdt:P106 wd:Q484188 .
?serialKiller wdt:P31 wd:Q5 .
?serialKiller wdt:P1345 ?victimCount .
?serialKiller wdt:P27 ?countryOfCitizenship .
?countryOfCitizenship wdt:P31 wd:Q3624078 .
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" .
    ?serialKiller rdfs:label ?serialKillerLabel .
    ?countryOfCitizenship rdfs:label ?countryOfCitizenshipLabel .
    }
}
ORDER BY DESC(?victimCount)
LIMIT 20
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Serial Killer': result['serialKillerLabel']['value'],
      'Victims': result['victimCount']['value'],
      'Country': result['countryOfCitizenshipLabel']['value']
    }
    for result in results['results']['bindings']
])

print(results)

                Serial Killer Victims                     Country
0               Samuel Little      93    United States of America
1                Gary Ridgway      49    United States of America
2             Salvatore Riina      45                       Italy
3                  Wang Qiang      45  People's Republic of China
4            Andrei Chikatilo      43                      Russia
5       Hadj Mohammed Mesfewi      36                     Morocco
6                   Ted Bundy      36    United States of America
7             François Vérove      25                      France
8     Ángel Maturino Reséndiz      23                      Mexico
9                 Yvan Keller      23                      France
10               Earle Nelson      22    United States of America
11              William Bonin      21    United States of America
12             Yoo Young-chul      20                 South Korea
13             Thierry Paulin      18                      France
14        

In [ ]:
query = """
SELECT ?countryLabel (COUNT(?serial_killer) AS ?count) WHERE {
  ?serial_killer wdt:P106 wd:Q484188 .
  ?serial_killer wdt:P31 wd:Q5 .
  ?serial_killer wdt:P27 ?country .
  ?country wdt:P31 wd:Q3624078 .
  ?country rdfs:label ?countryLabel .
  FILTER (LANG(?countryLabel) = "en")
}
GROUP BY ?countryLabel ?country
ORDER BY DESC(?count)
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Country': result['countryLabel']['value'],
      'Serial Killers': result['count']['value'],
    }
    for result in results['results']['bindings']
])

print(results)

                     Country Serial Killers
0   United States of America            280
1                     Russia             58
2                     France             49
3             United Kingdom             43
4                    Germany             33
..                       ...            ...
61               North Korea              1
62               New Zealand              1
63                 Venezuela              1
64                   Ecuador              1
65                   Bolivia              1

[66 rows x 2 columns]


In [ ]:
query = """
SELECT (COUNT(?male_serial_killer) AS ?male_count) (COUNT(?female_serial_killer) AS ?female_count) WHERE {
  {
    SELECT ?male_serial_killer WHERE {
      ?male_serial_killer wdt:P106 wd:Q484188 .
      ?male_serial_killer wdt:P31 wd:Q5 .
      ?male_serial_killer wdt:P21 wd:Q6581097 .
      ?male_serial_killer wdt:P31 wd:Q5 .
    }
  }
  UNION
  {
    SELECT ?female_serial_killer WHERE {
      ?female_serial_killer wdt:P106 wd:Q484188 .
      ?female_serial_killer wdt:P31 wd:Q5 .
      ?female_serial_killer wdt:P21 wd:Q6581072 .
      ?female_serial_killer wdt:P31 wd:Q5 .
    }
  }
}
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Males': result['male_count']['value'],
      'Females': result['female_count']['value']
    }
    for result in results['results']['bindings']
])

print(results)

  Males Females
0   946     134


In [ ]:
query = """
SELECT DISTINCT ?occupationLabel (COUNT(?serialKiller) AS ?count) WHERE {
?serialKiller wdt:P106 wd:Q484188;
wdt:P106 ?occupation .
?occupation wdt:P31 ?profession.
FILTER (?profession = wd:Q3922583 || ?profession = wd:Q28640 || ?profession = wd:Q192581 || ?profession = wd:Q4164871 || ?profession = wd:Q6958747 || ?profession = wd:Q1914636)
SERVICE wikibase:label {
bd:serviceParam wikibase:language "en".
?occupation rdfs:label ?occupationLabel .
}
}
GROUP BY ?occupationLabel
ORDER BY DESC(?count)
LIMIT 20
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Occupation': result['occupationLabel']['value'],
      'Count': result['count']['value']
    }
    for result in results['results']['bindings']
])

print(results)

            Occupation Count
0                nurse    31
1   military personnel    25
2            physician    18
3           prostitute     8
4               writer     8
5       police officer     8
6              soldier     8
7           journalist     8
8            scientist     8
9         psychiatrist     6
10            mechanic     6
11        truck driver     4
12      businessperson     4
13             butcher     4
14        entrepreneur     4
15                poet     4
16        photographer     4
17         electrician     4
18         taxi driver     4
19         salesperson     4


In [ ]:
query = """
  SELECT (AVG(?age) AS ?average_age) WHERE {
  ?serial_killer wdt:P106 wd:Q484188 .
  ?serial_killer wdt:P31 wd:Q5 .
  ?serial_killer wdt:P569 ?birth_date .
  ?serial_killer wdt:P570 ?death_date .
  BIND(YEAR(?death_date) - YEAR(?birth_date) AS ?age)
}
"""

sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Average Life Expectancy': result['average_age']['value']
    }
    for result in results['results']['bindings']
])

print(results)

   Average Life Expectancy
0  48.53530377668308702791


In [ ]:
query = """
SELECT DISTINCT ?penaltyLabel (COUNT(?person) AS ?count)
WHERE {
  ?person wdt:P106 wd:Q484188 ;
          wdt:P1596 ?penalty .
  ?penalty rdfs:label ?penaltyLabel .
  FILTER (LANG(?penaltyLabel) = "en")
}
GROUP BY ?penaltyLabel
"""

sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
      'Penalty': result['penaltyLabel']['value'],
      'Count': result['count']['value']
    }
    for result in results['results']['bindings']
])

print(results)

                        Penalty Count
0                 dismemberment     1
1             life imprisonment   111
2   back-to-back life sentences     1
3          solitary confinement     1
4   Life imprisonment in Russia     1
5          penal transportation     1
6                       hanging     2
7        involuntary commitment     8
8                 incarceration     6
9   capital punishment in Japan     3
10         mandatory sentencing     1
11                         None     1
12           capital punishment   148
13               electric chair     1
14                   immurement     1
15                 imprisonment     1
16         execution by hanging     1
17                      Katorga     1


In [ ]:
query = """
SELECT 
  (COUNT(DISTINCT ?personWithEducation) / COUNT(DISTINCT ?person) * 100 AS ?percentWithEducation)
  (COUNT(DISTINCT ?personWithoutEducation) / COUNT(DISTINCT ?person) * 100 AS ?percentWithoutEducation)
WHERE {
{
    SELECT DISTINCT ?person
    WHERE {
      ?person wdt:P106 wd:Q484188 ;
              wdt:P69 ?education .
    }
  }
  BIND (COUNT(DISTINCT ?person) AS ?total) .
  {
    SELECT DISTINCT ?personWithEducation
    WHERE {
      ?personWithEducation wdt:P106 wd:Q484188 ;
                          wdt:P69 ?education .
      ?education wdt:P31/wdt:P279* wd:Q3918 .
    }
  }
  BIND (COUNT(DISTINCT ?personWithEducation) AS ?withEducation) .
  {
    SELECT DISTINCT ?personWithoutEducation
    WHERE {
      ?personWithoutEducation wdt:P106 wd:Q484188 ;
                             wdt:P69 ?education .
      FILTER NOT EXISTS { ?education wdt:P31/wdt:P279* wd:Q3918 . }
    }
  }
  BIND (COUNT(DISTINCT ?personWithoutEducation) AS ?withoutEducation) .
}
"""

sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
        'With Education' : result['percentWithEducation']['value'],
        'Without Education' : result['percentWithoutEducation']['value']
    }
    for result in results['results']['bindings']
])

print(results)

         With Education      Without Education
0  35.77981651376146789  83.486238532110091743


In [ ]:
query = """
SELECT ?conditionLabel (COUNT(?condition) AS ?count) WHERE {
  ?person wdt:P106 wd:Q484188 .
  ?person wdt:P106 wd:Q484188.
  ?person wdt:P1050 ?condition .
  ?condition wdt:P1995 ?healthSpecialty .
FILTER (?healthSpecialty IN (wd:Q7867, wd:Q9418, wd:Q170912, wd:Q178059))
SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?condition rdfs:label ?conditionLabel .
  }
}
GROUP BY ?conditionLabel
ORDER BY DESC(?count)
"""

sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

results = pd.DataFrame([
    {
        'Mental Ilness' : result['conditionLabel']['value'],
        'Count' : result['count']['value']
    }
    for result in results['results']['bindings']
])

print(results)

                                   Mental Ilness Count
0                antisocial personality disorder    18
1                borderline personality disorder     6
2                                     pedophilia     4
3                                  schizophrenia     3
4                               bipolar disorder     3
5                                    kleptomania     2
6                           erectile dysfunction     2
7                           personality disorder     2
8                                     alcoholism     2
9                                    necrophilia     2
10  personality disorder not otherwise specified     2
11                        sexual sadism disorder     1
12                               speech disorder     1
13                                   psychopathy     1
14                                      muteness     1
15                                     psychosis     1
